# Brick optimisation - simple fermionic cases

Created 10/102/2025

Objectives:
* Sweep over all simple fermionic cases, checking to see if we can extract the SPT and the extraction makes sense.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from functools import reduce
from operator import mul

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [233]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

from SPTOptimization.gradients import (
    expectation_gradient_from_environments_and_b_tensor
)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [9]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [10]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x14ff1b850>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x153975ed0>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x154f1d690>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x1532abc90>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x154fe76d0>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x15596cd50>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x1532a9f10>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x1559a7350>}

In [11]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 4.10310779e-55, 3.20108804e-32, 4.41166527e-30,
       4.30652209e-31, 1.28028128e-32, 2.18979328e-32, 9.64766348e-32,
       6.39204896e-31, 4.19476267e-32, 7.51835939e-33, 1.56998498e-32,
       3.06132263e-29, 6.24832887e-32, 6.64529320e-32, 6.61192056e-33,
       3.02331734e-32, 1.97077344e-32, 3.97848871e-32, 1.65936555e-32,
       8.12962747e-32, 8.36180681e-32, 1.05455111e-32, 2.30695596e-32,
       4.15719467e-32, 3.32855567e-32, 1.02762007e-32, 1.46697913e-32,
       3.80115562e-31, 1.63756131e-32, 9.98608231e-33, 1.82981182e-32,
       1.73380216e-33, 6.00451123e-32, 1.33036658e-32, 2.79091135e-31,
       1.64291033e-32, 3.68006128e-33, 9.95622775e-32, 6.17421509e-32,
       2.70280235e-32, 3.01371615e-31, 2.35183074e-32, 1.17896355e-32,
       3.93686429e-31, 8.45721273e-32, 1.08863312e-32, 3.77353007e-31,
       9.67562785e-33, 1.99656454e-33, 6.40393616e-33, 2.78084102e-32,
       1.03150150e-31, 4.37579428e-33, 1.45445289e-32, 3.55938597e-32,
      

In [12]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 3.26915052e-29, 3.33293732e-29, 0.00000000e+00,
       3.33293732e-29, 7.79030959e-29, 4.21824880e-29, 4.21824880e-29,
       5.96576060e-30, 1.14662165e-29, 3.59424750e-29, 8.18720523e-29,
       4.21824880e-29, 8.28796989e-29, 4.21824880e-29, 3.79670126e-29,
       4.21824880e-29, 8.18720523e-29, 1.37190923e-28, 0.00000000e+00,
       1.37190923e-28, 0.00000000e+00, 3.66111579e-29, 3.66111579e-29,
       3.86541844e-29, 3.33293732e-29, 0.00000000e+00, 0.00000000e+00,
       3.59424750e-29, 1.56000325e-28, 3.46235982e-29, 0.00000000e+00,
       3.59424750e-29, 3.59424750e-29, 3.72860037e-29, 0.00000000e+00,
       0.00000000e+00, 9.22258517e-29, 8.33234331e-30, 9.01057880e-29,
       3.46235982e-29, 4.43734259e-29, 3.14342582e-29, 3.14342582e-29,
       0.00000000e+00, 8.18720523e-29, 1.46709639e-29, 1.46709639e-29,
       1.46709639e-29, 8.98561875e-30, 0.00000000e+00, 8.01495006e-30,
       4.07526776e-29, 4.00470169e-29, 9.32150093e-30, 7.69262642e-29,
      

In [376]:
loaded_exps = dict()

for k, psi in loaded_data.items():
    exp = psi.expectation_value('N', sites=[101,])[0]
    loaded_exps[k] = exp

In [377]:
loaded_exps

{(False, (1, 1)): 0.4999999999999988,
 (True, (1, 1)): 0.4999999999999729,
 (True, (0, 1)): 0.49999999999999745,
 (False, (0, 1)): 0.4999999999999949,
 (False, (1, 0)): 0.49999999999998695,
 (True, (1, 0)): 0.49999999999999367,
 (True, (0, 0)): 6.403936160073908e-33,
 (False, (0, 0)): 0.0}

# Definitons

In [13]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [14]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [15]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [16]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [17]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [18]:
fermionic_np_symmetries = [np_I, np_JW]

In [19]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [20]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [21]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [22]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [23]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [24]:
shifted_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(shifted_symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I]*1,
                right_boundary_unitaries=[np_I, np_00]*1
            )

            shifted_cases[(*k, i, j)] = case

In [25]:
for c in shifted_cases.values():
    c.compute_svd_approximate_expectation()

In [26]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [27]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions

### Leg and label functions

In [28]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def is_single_physical_leg_label(label):
    return bool(p_leg_pattern.match(label))

In [29]:
def is_grouped_physical_leg_label(leg_label):
    head, *body, tail = leg_label
    if (head != '(') or (tail != ')'):
        return False

    body = ''.join(body)
    legs = body.split('.')

    return all(is_single_physical_leg_label(l) for l in legs)

In [30]:
def is_physical_leg_label(label):
    out = (
        is_single_physical_leg_label(label)
        or is_grouped_physical_leg_label(label)
    )

    return out

In [31]:
def extract_single_physical_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if is_single_physical_leg_label(l)
    )

    return out

In [32]:
def conjugate_single_physical_leg_label(label):
    return label + '*'

In [33]:
def conjugate_grouped_physical_leg_label(label):
    _, *body, _ = label
    single_labels = ''.join(body).split('.')

    conjugate_labels = [
        conjugate_single_physical_leg_label(l)
        for l in single_labels
    ]

    conjugate_body = '.'.join(conjugate_labels)
    out = '(' + conjugate_body + ')'
    
    return out

In [34]:
def conjugate_leg_label(label):
    if is_single_physical_leg_label(label):
        return conjugate_single_physical_leg_label(label)
    elif is_grouped_physical_leg_label(label):
        return conjugate_grouped_physical_leg_label(label)
    else:
        raise ValueError

In [35]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

### Other

In [36]:
def contract_virtual_legs(tl, tr):
    return npc.tensordot(tl, tr, ['vR', 'vL'])

In [37]:
def combine_tensors(tensors):
    out = reduce(contract_virtual_legs, tensors)

    leg_labels = [
        extract_single_physical_leg_label_from_tensor(t)
        for t in tensors
    ]

    out = out.combine_legs(leg_labels)

    return out

In [38]:
def combine_b_tensors(b_tensors):
    renamed_tensors = [
        b.replace_label('p', f'p{i}')
        for i, b in enumerate(b_tensors)
    ]

    return combine_tensors(renamed_tensors)

In [39]:
def get_identity_operator(mps_tensor):
    p_leg_label = get_physical_leg_labels(mps_tensor)[0]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [40]:
def svd_reduce_split_tensor(t, max_inner_dim=MAX_VIRTUAL_BOND_DIM,
                           normalise=True, svd_cutoff=SVD_CUTOFF):
    U, S, VH = npc.svd(
        t,
        compute_uv=True,
        inner_labels=['vR', 'vL'],
        cutoff=svd_cutoff
    )

    # Truncate tensors:
    U = U[:, :max_inner_dim]
    S = S[:max_inner_dim]
    VH = VH[:max_inner_dim, :]

    if normalise:
        new_norm = np.sqrt(np.sum(S**2))
        S = S/new_norm

    """
    leg = VH.get_leg('vL')

    schmidt_values = npc.diag(S, leg, labels=['vL', 'vR'])
    """

    return U, S, VH

In [41]:
def split_combined_b(b, leftmost_schmidt_values,
                     max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = b.split_legs()

    num_sites = t.ndim - 2

    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    out_bs = list()
    out_schmidt_values = list()

    current_left_schmidt_values = leftmost_schmidt_values

    for i, ll in enumerate(p_leg_labels[:-1]):
        # In case the bond dimension has been truncated. May need to add in a
        # case if have less schmidt values than the bond dim...
        bond_dim = t.get_leg('vL').get_block_sizes()[0]
        t.iscale_axis(current_left_schmidt_values[:bond_dim], axis='vL')

        tail_legs = p_leg_labels[(i+1):]
        
        t = t.combine_legs([['vL', ll], ['vR', *tail_legs]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=True
        )

        bl = (
            U
            .split_legs()
            .replace_label(ll, 'p')
        )
        bl.iscale_axis(1/current_left_schmidt_values[:bond_dim], axis='vL')
        bl.iscale_axis(S, axis='vR')
        bl.itranspose(['vL', 'p', 'vR'])
        out_bs.append(bl)

        out_schmidt_values.append(S)
        current_left_schmidt_values=S

        t = VH.split_legs()

    bl = t.replace_label(p_leg_labels[-1], 'p')
    bl.itranspose(['vL', 'p', 'vR'])
    out_bs.append(bl)

    return out_bs, out_schmidt_values

In [42]:
def split_b(b, max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    leg_label = get_physical_leg_labels(b)[0]

    if is_single_physical_leg_label(leg_label):
        return b
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_b(b, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [43]:
def split_combined_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = u.split_legs()
    num_sites = t.ndim //2
    
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    p_leg_labels_hc = [l+'*' for l in p_leg_labels]
    out_ws = list()

    for i, (ll, ll_hc) in enumerate(zip(p_leg_labels[:-1], p_leg_labels_hc[:-1])):
        tail_legs = p_leg_labels[(i+1):]
        tail_legs_hc = p_leg_labels_hc[(i+1):]

        left_leg_labels = [ll, ll_hc]
        if 'vL' in t.get_leg_labels():
            left_leg_labels.append('vL')
    
        t = t.combine_legs([left_leg_labels, [*tail_legs, *tail_legs_hc]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=False
        )

        U.iscale_axis(S, axis='vR')

        wl = U.split_legs().replace_labels([ll, ll_hc], ['p', 'p*'])
        #wl.itranspose(['p', 'p*'])
        out_ws.append(wl)

        t = VH.split_legs()

    wl = t.replace_labels(
        [p_leg_labels[-1], p_leg_labels_hc[-1]],
        ['p', 'p*']
    )
    #wl.itranspose(['p', 'p*'])
    out_ws.append(wl)

    out_ws[0] = (
        out_ws[0]
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    out_ws[-1] = (
        out_ws[-1]
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out_ws

In [44]:
def split_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
            p_leg_labels=None):
    leg_label = get_physical_leg_labels(u)[0]

    if is_single_physical_leg_label(leg_label):
        return [unitary_to_mpo_tensor(u),]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_u(u, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [45]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [46]:
def multiply_individual_mpo_tensors(w1, w2):
    w1 = w1.replace_labels(['vR', 'vL'], ['vR1', 'vL1'])
    w2 = w2.replace_labels(['vR', 'vL'], ['vR2', 'vL2'])

    w = (
        npc.tensordot(w1, w2, [['p'], ['p*']])
        .combine_legs([['vR1', 'vR2'], ['vL1', 'vL2']])
        .replace_labels(['(vR1.vR2)', '(vL1.vL2)'], ['vR', 'vL'])
    )

    return w

In [47]:
def group_elements(l, group_size, offset=0):
    first, rest = l[:offset], l[offset:]

    num_rest_groups = ((len(rest)-1)//group_size) + 1

    groups = [first,] if first else list()

    for i in range(num_rest_groups):
        first_index = i*group_size
        last_index = (i+1)*group_size
        groups.append(rest[first_index:last_index])

    return groups

In [48]:
def combine_grouped_b_tensors(grouped_bs):
    out = list()

    for group in grouped_bs:
        if len(group) == 1:
            out.append(group[0])
        else:
            out.append(combine_b_tensors(group))

    return out

In [49]:
def reduce_virtual_mpo_dim(wl, wr, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    # wl = wl.replace_labels(['p', 'p*'], ['pl', 'pl*'])

    wl = wl.combine_legs(['p', 'vL', 'p*'])
    wr = wr.combine_legs(['p', 'vR', 'p*'])
    w = npc.tensordot(wl, wr, ['vR', 'vL'])

    wl, S, wr = svd_reduce_split_tensor(
        w,
        max_inner_dim=max_virtual_bond_dim
    )

    wl.iscale_axis(S, axis='vR')

    wl = wl.split_legs()
    wr = wr.split_legs()

    return wl, wr

In [50]:
def multiply_mpos(top_ws, bottom_ws, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    ws = list()
    
    for w1, w2 in zip(top_ws, bottom_ws):
        w = multiply_individual_mpo_tensors(w1, w2)
        ws.append(w)


    for i in range(len(ws)-1):
        ws[i], ws[i+1] = reduce_virtual_mpo_dim(
            ws[i],
            ws[i+1],
            max_virtual_bond_dim=max_virtual_bond_dim
        )

    return ws

### Layer function

In [51]:
def optimise_layer(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values

In [52]:
def inner_product_b_tensors(b_tensors, b_bra_tensors=None, left_environment=None,
                            right_environment=None):
    if b_bra_tensors is None:
        b_bra_tensors = b_tensors

    b = b_tensors[0]
    b_bra = b_bra_tensors[0]

    if left_environment is None:
        t = npc.tensordot(b, b_bra.conj(), [['vL',], ['vL*',]])
    else:
        t = npc.tensordot(left_environment, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    for b, b_bra in zip(b_tensors[1:], b_bra_tensors[1:]):
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    if right_environment is None:
        out = npc.trace(t)
    else:
        out = npc.tensordot(t, right_environment, [['vR', 'vR*'], ['vL', 'vL*']])

    return out

In [53]:
def get_left_side_right_symmetry_environment(
    right_top_b_tensors, right_bottom_b_tensors, symmetry_transfer_matrix
    ):

    if right_bottom_b_tensors is None:
        right_bottom_b_tensors = right_top_b_tensors

    t = get_right_identity_environment_from_tp_tensor(right_top_b_tensors[-1])

    for tb, bb in zip(right_top_b_tensors[::-1], right_bottom_b_tensors[::-1]):
        t = npc.tensordot(t, tb, [['vL',], ['vR']])
        t = npc.tensordot(t, bb.conj(), [['vL*', 'p'], ['vR*', 'p*']])

    t = npc.tensordot(
        t,
        symmetry_transfer_matrix,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    return t

In [54]:
def swap_left_right_indices(npc_array):
    left_right_pairs = {
        'vL': 'vR',
        'vR': 'vL',
        'vL*': 'vR*',
        'vR*': 'vL*'
    }

    leg_labels = npc_array.get_leg_labels()

    old_labels = [l for l in leg_labels if l in left_right_pairs]
    new_labels = [left_right_pairs[l] for l in old_labels]

    out = npc_array.replace_labels(old_labels, new_labels)

    return out

In [55]:
def two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_pair = optimise_layer(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_pair[0]
    new_right_side_left_schmidt_values = new_right_pair[1]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_pair = optimise_layer(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_pair[0]
    new_left_side_right_schmidt_values = new_left_pair[1]

    return (new_left_pair, new_right_pair)

In [56]:
def initialize_brick_optimisation(symmetry_case, num_sites):
    symmetry_transfer_matrix = symmetry_case.npc_symmetry_transfer_matrix

    right_site_indices = list(range(
        symmetry_case.right_symmetry_index + 1,
        symmetry_case.right_symmetry_index + 1 + num_sites
    ))

    right_mps_tensors = [
        symmetry_case.psi.get_B(i)
        for k, i in enumerate(right_site_indices)
    ]

    right_side_left_schmidt_values = [
        symmetry_case.psi.get_SL(i)
        for i in right_site_indices
    ]

    left_site_indices = list(range(
        symmetry_case.left_symmetry_index - 1,
        symmetry_case.left_symmetry_index - 1 - num_sites,
        -1
    ))

    left_mps_tensors = [
        symmetry_case.psi.get_B(i, form='A')
        for k, i in enumerate(left_site_indices)
    ]
    
    left_mps_tensors = [swap_left_right_indices(b) for b in left_mps_tensors]

    left_side_right_schmidt_values = [
        symmetry_case.psi.get_SR(i)
        for i in left_site_indices
    ]

    right_expectations=list()
    left_expectations=list()
    left_unitaries=list()
    right_unitaries=list()

    return (
        symmetry_transfer_matrix,
        left_mps_tensors,
        right_mps_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        right_expectations,
        left_expectations,
        left_unitaries,
        right_unitaries
    )

### Output gradient tensor "one site" functions

In [241]:
def optimise_layer_with_gradient(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    # Below code is a hack to add in the gradient, assuming one block.
    right_environment = get_right_identity_environment_from_tp_tensor(
        top_b_tensors[-1]
    )

    grad = expectation_gradient_from_environments_and_b_tensor(
        top_combined_bs[0],
        left_environment,
        right_environment,
        bottom_combined_bs[0]
    )
    # Hack ends here.

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values, grad

In [237]:
def two_sided_optimise_layer_with_gradient(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_triple = optimise_layer_with_gradient(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_triple[0]
    new_right_side_left_schmidt_values = new_right_triple[1]
    new_right_grad = new_right_triple[2]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_triple = optimise_layer_with_gradient(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_triple[0]
    new_left_side_right_schmidt_values = new_left_triple[1]
    new_left_grad = new_left_triple[2]

    return (new_left_triple, new_right_triple)

### Output calculated symmetry environments

In [304]:
def two_sided_optimise_layer_with_environments(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_triple = optimise_layer_with_gradient(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_triple[0]
    new_right_side_left_schmidt_values = new_right_triple[1]
    new_right_grad = new_right_triple[2]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_triple = optimise_layer_with_gradient(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_triple[0]
    new_left_side_right_schmidt_values = new_left_triple[1]
    new_left_grad = new_left_triple[2]

    return (
        (*new_left_triple, left_side_right_symmetry_environment),
        (*new_right_triple, right_side_left_symmetry_environment)
    )

# Test

In [57]:
cases.keys()

dict_keys([(False, (1, 1), 0, 0), (False, (1, 1), 0, 1), (False, (1, 1), 0, 2), (False, (1, 1), 0, 3), (False, (1, 1), 1, 0), (False, (1, 1), 1, 1), (False, (1, 1), 1, 2), (False, (1, 1), 1, 3), (True, (1, 1), 0, 0), (True, (1, 1), 0, 1), (True, (1, 1), 0, 2), (True, (1, 1), 0, 3), (True, (1, 1), 1, 0), (True, (1, 1), 1, 1), (True, (1, 1), 1, 2), (True, (1, 1), 1, 3), (True, (0, 1), 0, 0), (True, (0, 1), 0, 1), (True, (0, 1), 0, 2), (True, (0, 1), 0, 3), (True, (0, 1), 1, 0), (True, (0, 1), 1, 1), (True, (0, 1), 1, 2), (True, (0, 1), 1, 3), (False, (0, 1), 0, 0), (False, (0, 1), 0, 1), (False, (0, 1), 0, 2), (False, (0, 1), 0, 3), (False, (0, 1), 1, 0), (False, (0, 1), 1, 1), (False, (0, 1), 1, 2), (False, (0, 1), 1, 3), (False, (1, 0), 0, 0), (False, (1, 0), 0, 1), (False, (1, 0), 0, 2), (False, (1, 0), 0, 3), (False, (1, 0), 1, 0), (False, (1, 0), 1, 1), (False, (1, 0), 1, 2), (False, (1, 0), 1, 3), (True, (1, 0), 0, 0), (True, (1, 0), 0, 1), (True, (1, 0), 0, 2), (True, (1, 0), 0, 3

In [58]:
test_case = cases[(False, (0,0), 0, 0)]

In [59]:
(
    symmetry_transfer_matrix,
    left_mps_tensors,
    right_mps_tensors,
    right_side_left_schmidt_values,
    left_side_right_schmidt_values,
    right_expectations,
    left_expectations,
    left_unitaries,
    right_unitaries
) = initialize_brick_optimisation(test_case, 2)

In [60]:
block_width = 2
num_iterations = 1
num_layers = 1

top_right_mpo_tensors = [right_mps_tensors,]
top_left_mpo_tensors = [left_mps_tensors,]

right_side_left_schmidt_values_list = [right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [left_side_right_schmidt_values,]

In [61]:
symmetry_transfer_matrix

<npc.Array shape=(1, 1, 1, 1) labels=['vL*', 'vL', 'vR', 'vR*']>

In [62]:
left_mps_tensors

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [63]:
right_mps_tensors

[<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(1, 2, 1) labels=['vL', 'p', 'vR']>]

In [64]:
right_side_left_schmidt_values

[array([1.]), array([1.])]

In [65]:
left_side_right_schmidt_values

[array([1.]), array([1.])]

In [66]:
top_left_mpo_tensors[-1]

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [67]:
left_mps_tensors

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [68]:
for i in range(num_layers):
    print(f"Layer: {i}")
    
    block_offset = (block_width//2)*i % block_width

    out_quad = two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_mpo_tensors[-1],
        top_right_mpo_tensors[-1],
        right_side_left_schmidt_values_list[-1],
        left_side_right_schmidt_values_list[-1],
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        left_mps_tensors,
        right_mps_tensors,
        left_unitaries,
        right_unitaries,
        num_iterations
    )

    left_pair, right_pair = out_quad

    top_right_mpo_tensors.append(right_pair[0])
    right_side_left_schmidt_values_list.append(right_pair[1])
    
    top_left_mpo_tensors.append(left_pair[0])
    left_side_right_schmidt_values_list.append(left_pair[1])

Layer: 0


In [69]:
left_expectations

[[[0.9999999999999977]]]

# Sweep

In [70]:
block_width = 2
num_iterations = 1
num_layers = 1
num_sites = 2

In [71]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])


    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

## JW assumption check

In [72]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [73]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [74]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [75]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [76]:
def expect_jw_necessary(n1_pair, symmetry_index):
    n1_func = get_n1_func(*n1_pair)

    return bool(n1_func(symmetry_index, 0))

In [77]:
expect_jw_necessary((1,0), 2)

False

In [78]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [79]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((9.65745524570475e-18, 1, True, True),
 (8.367014067204974e-14, 0, True, False))

In [80]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [81]:
(
    expected_left_expecations[(False, (1, 1), 1, 1)],
    expected_left_expecations[(False, (1, 1), 0, 1)]
)

((1.0000000000000022, 1, True, True), (0.9999999942648465, 0, True, False))

# Sweep over on 4 boundary sites

## Fermion parity commutation/anticommutation

In [175]:
test_unitary = right_unitaries[(False, (0,0), 0, 0)]
test_unitary = test_unitary[0][0]

In [176]:
leg_labels = ['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*', 'p3', 'p3*']
legs_charges = [
    test_unitary.split_legs().get_leg(l)
    for l in leg_labels
]

In [369]:
X = np_00[(...,)  + (np.newaxis,)*2]*np_JW[(np.newaxis,)*2 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_00[(np.newaxis,)*4 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_JW[(np.newaxis,)*6 + (...,)]

In [178]:
right_boundary_jw = npc.Array.from_ndarray(
    X,
    legs_charges,
    dtype=np.complex128,
    labels=leg_labels
)

right_boundary_jw = right_boundary_jw.combine_legs(
    [['p0', 'p1', 'p2', 'p3'], ['p0*', 'p1*', 'p2*', 'p3*']]
)

In [179]:
X = np_JW[(...,)  + (np.newaxis,)*2]*np_00[(np.newaxis,)*2 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_JW[(np.newaxis,)*4 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_00[(np.newaxis,)*6 + (...,)]

In [180]:
left_test_unitary = left_unitaries[(False, (0,0), 0, 0)]
left_test_unitary = left_test_unitary[0][0]

In [181]:
left_legs_charges = [
    left_test_unitary.split_legs().get_leg(l)
    for l in leg_labels
]

In [182]:
left_boundary_jw = npc.Array.from_ndarray(
    X,
    left_legs_charges,
    dtype=np.complex128,
    labels=leg_labels
)

left_boundary_jw = left_boundary_jw.combine_legs(
    [['p0', 'p1', 'p2', 'p3'], ['p0*', 'p1*', 'p2*', 'p3*']]
)

In [189]:
np.linalg.norm

<function norm at 0x104b55e70>

In [403]:
def fermion_parity_operator_check(operator, fermion_parity_operator,
                                  num_sites=4):
    leg_labels = [f'p{i}' for i in range(num_sites)]
    leg_labels_conj = [s + '*' for s in leg_labels]

    joined_leg_label = '(' + '.'.join(leg_labels) + ')'
    joined_leg_label_conj = '(' + '.'.join(leg_labels_conj) + ')'

    op = (
        operator
        .transpose([joined_leg_label, joined_leg_label_conj])
        .to_ndarray()
    )
    fp = fermion_parity_operator.to_ndarray()

    transformed_op = fp @ op @ fp

    x = np.trace(transformed_op @ (op.T.conj()))
    phase = x/np.abs(x)

    assert np.isreal(phase)
    phase = np.real(phase)

    error = np.linalg.norm(
        phase*transformed_op - op,
        ord='fro'
    )

    return (phase, error)

## Sweep

In [196]:
block_width = 4
num_iterations = 1
num_layers = 1
num_sites = 4

In [210]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()
left_phase_errors = dict()
right_phase_errors = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])

        # Only works for one layer, one block unitaries...
        left_phase_error_pair = fermion_parity_operator_check(
            current_left_unitaries[0][0],
            left_boundary_jw
        )
        right_phase_error_pair = fermion_parity_operator_check(
            current_right_unitaries[0][0],
            right_boundary_jw
        )

    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries
    left_phase_errors[k] = left_phase_error_pair
    right_phase_errors[k] = right_phase_error_pair

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

## JW assumption check

In [84]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [85]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((2.1610706909084384e-17, 1, True, True),
 (8.350306722905389e-14, 0, True, False))

In [86]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [87]:
list(left_expectations.keys())[0]

(False, (1, 1), 0, 0)

In [211]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[0][0][0],
        vr[0][0][0],
        exp,
        b1,
        b2,
        *left_phase_errors[k],
        *right_phase_errors[k],
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool',
        'left_jw_phase',
        'left_jw_error',
        'right_jw_phase',
        'right_jw_error',
    ]
)

In [212]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,left_jw_phase,left_jw_error,right_jw_phase,right_jw_error
0,False,1,1,0,0,1.0,1.000000e+00,1,False,False,1.0,9.974883,1.0,9.839742
1,False,1,1,0,1,1.0,8.350307e-14,0,True,False,-1.0,9.899388,-1.0,11.051460
2,False,1,1,0,2,1.0,8.339337e-14,0,True,False,-1.0,10.204176,-1.0,11.273186
3,False,1,1,0,3,1.0,1.000000e+00,1,False,False,1.0,9.280659,1.0,10.497549
4,False,1,1,1,0,1.0,1.892791e-14,0,False,True,1.0,10.170507,1.0,10.655641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.000000e+00,1,False,False,1.0,7.900571,1.0,9.508861
60,False,0,0,1,0,1.0,1.000000e+00,0,False,True,1.0,7.353676,1.0,10.065557
61,False,0,0,1,1,1.0,1.000000e+00,0,False,True,1.0,8.019910,1.0,10.274669
62,False,0,0,1,2,1.0,1.000000e+00,0,False,True,1.0,7.814812,1.0,8.014084


In [213]:
expectations_df['left_jw_error'].describe()

count    64.000000
mean      8.691927
std       2.303631
min       2.816355
25%       8.338984
50%       9.496207
75%      10.127175
max      10.964077
Name: left_jw_error, dtype: float64

In [214]:
expectations_df['right_jw_error'].describe()

count    64.000000
mean      9.399459
std       1.474549
min       5.743955
25%       9.253907
50%       9.657611
75%      10.339312
max      11.273186
Name: right_jw_error, dtype: float64

In [223]:
expectations_df['left_jw_phase'] = expectations_df['left_jw_phase'].round()

In [224]:
expectations_df['left_jw_phase'].value_counts()

left_jw_phase
 1.0    40
-1.0    24
Name: count, dtype: int64

In [225]:
expectations_df['right_jw_phase'] = expectations_df['right_jw_phase'].round()

In [226]:
expectations_df['right_jw_phase'].round().value_counts()

right_jw_phase
 1.0    40
-1.0    24
Name: count, dtype: int64

In [227]:
(expectations_df['right_jw_phase']*expectations_df['left_jw_phase']).round().value_counts()

1.0    64
Name: count, dtype: int64

In [228]:
expectations_df[['predicted_jw_necessary', 'left_jw_phase', 'right_jw_phase']].value_counts()

predicted_jw_necessary  left_jw_phase  right_jw_phase
False                    1.0            1.0              40
True                    -1.0           -1.0              24
Name: count, dtype: int64

In [91]:
expectations_df['left_expectation'].describe()

count    64.000000
mean      0.999996
std       0.000017
min       0.999901
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: left_expectation, dtype: float64

In [230]:
(
    expectations_df[~expectations_df['jw_string_present_bool']]
)['left_expectation']

0     1.000000
1     1.000000
2     1.000000
3     1.000000
8     1.000000
9     1.000000
10    1.000000
11    1.000000
16    1.000000
17    0.999994
18    1.000000
19    1.000000
24    1.000000
25    1.000000
26    1.000000
27    1.000000
32    1.000000
33    1.000000
34    1.000000
35    1.000000
40    1.000000
41    0.999990
42    0.999999
43    1.000000
48    1.000000
49    0.999935
50    0.999998
51    0.999998
56    1.000000
57    1.000000
58    1.000000
59    1.000000
Name: left_expectation, dtype: float64

In [232]:
(
    expectations_df[expectations_df['jw_string_present_bool']]
)['left_expectation']

4     1.000000
5     1.000000
6     1.000000
7     1.000000
12    1.000000
13    1.000000
14    1.000000
15    1.000000
20    1.000000
21    0.999999
22    1.000000
23    1.000000
28    1.000000
29    1.000000
30    1.000000
31    1.000000
36    1.000000
37    1.000000
38    1.000000
39    1.000000
44    0.999989
45    1.000000
46    0.999901
47    1.000000
52    1.000000
53    0.999935
54    0.999998
55    0.999998
60    1.000000
61    1.000000
62    1.000000
63    1.000000
Name: left_expectation, dtype: float64

In [231]:
(
    expectations_df[~expectations_df['jw_string_present_bool']]
)['right_expectation']

0     1.000000e+00
1     8.350307e-14
2     8.339337e-14
3     1.000000e+00
8     1.000000e+00
9     1.828361e-14
10    1.484096e-14
11    3.263023e-14
16    1.000000e+00
17    2.250570e-14
18    1.173689e-14
19    1.687474e-14
24    1.000000e+00
25    1.000000e+00
26    1.269122e-14
27    1.269122e-14
32    1.000000e+00
33    3.946076e-14
34    1.000000e+00
35    3.948722e-14
40    1.000000e+00
41    9.912543e-16
42    1.941947e-14
43    7.802992e-15
48    1.000000e+00
49    3.211215e-15
50    1.485644e-14
51    6.391326e-15
56    1.000000e+00
57    1.000000e+00
58    1.000000e+00
59    1.000000e+00
Name: right_expectation, dtype: float64

In [92]:
expectations_df['right_expectation_rounded'] = expectations_df['right_expectation'].round()
expectations_df['left_expectation_rounded'] = expectations_df['left_expectation'].round()

In [93]:
expectations_df['correct'] = (
    (expectations_df['right_expectation_rounded'] == expectations_df['predicted_expectation']) &
    (expectations_df['left_expectation_rounded'] == expectations_df['predicted_expectation'])
)

In [94]:
expectations_df['correct'].value_counts()

correct
False    50
True     14
Name: count, dtype: int64

In [95]:
expectations_df[expectations_df['correct']]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,1.0,True
8,True,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
16,True,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
24,False,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
25,False,0,1,0,1,1.0,1.0,1,False,False,1.0,1.0,True
32,False,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
34,False,1,0,0,2,1.0,1.0,1,False,False,1.0,1.0,True
40,True,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
48,True,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True


In [96]:
expectations_df[
    (~expectations_df['correct'])
    & (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
60,False,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
61,False,0,0,1,1,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
62,False,0,0,1,2,1.000000,1.000000e+00,0,False,True,1.0,1.0,False


In [97]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
48,True,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
56,False,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
57,False,0,0,0,1,1.000000,1.000000e+00,1,False,False,1.0,1.0,True


In [98]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
    & (expectations_df['right_expectation_rounded'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False


# Sweep over on 4 boundary sites with gradients

## Sweep

In [253]:
block_width = 4
num_iterations = 3
num_layers = 1
num_sites = 4

In [254]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()
left_phase_errors = dict()
right_phase_errors = dict()
right_grads = dict()
left_grads = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out = two_sided_optimise_layer_with_gradient(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_triple, right_triple = out
    
        top_right_mps_tensors.append(right_triple[0])
        right_side_left_schmidt_values_list.append(right_triple[1])
        
        top_left_mps_tensors.append(left_triple[0])
        left_side_right_schmidt_values_list.append(left_triple[1])

        # Only works for one layer, one block unitaries...
        left_phase_error_pair = fermion_parity_operator_check(
            current_left_unitaries[0][0],
            left_boundary_jw
        )
        right_phase_error_pair = fermion_parity_operator_check(
            current_right_unitaries[0][0],
            right_boundary_jw
        )

    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries
    left_phase_errors[k] = left_phase_error_pair
    right_phase_errors[k] = right_phase_error_pair
    left_grads[k] = left_triple[2]
    right_grads[k] = right_triple[2]

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

## JW assumption check

In [255]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    right_grad = right_grads[k]
    _, rs, _ = npc.svd(right_grad)
    right_svd_exp = np.sum(rs)
    
    left_grad = left_grads[k]
    _, ls, _ = npc.svd(left_grad)
    left_svd_exp = np.sum(ls)

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[0][0][0],
        vr[0][0][0],
        exp,
        b1,
        b2,
        *left_phase_errors[k],
        *right_phase_errors[k],
        left_svd_exp,
        right_svd_exp
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool',
        'left_jw_phase',
        'left_jw_error',
        'right_jw_phase',
        'right_jw_error',
        'left_svd_exp',
        'right_svd_exp'
    ]
)

In [256]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,left_jw_phase,left_jw_error,right_jw_phase,right_jw_error,left_svd_exp,right_svd_exp
0,False,1,1,0,0,1.0,1.000000e+00,1,False,False,1.0,9.974883,1.0,9.839742,1.0,1.000000e+00
1,False,1,1,0,1,1.0,8.350307e-14,0,True,False,-1.0,9.899388,-1.0,11.051460,1.0,8.350307e-14
2,False,1,1,0,2,1.0,8.339337e-14,0,True,False,-1.0,10.204176,-1.0,11.273186,1.0,8.339337e-14
3,False,1,1,0,3,1.0,1.000000e+00,1,False,False,1.0,9.280659,1.0,10.497549,1.0,1.000000e+00
4,False,1,1,1,0,1.0,1.892791e-14,0,False,True,1.0,10.170507,1.0,10.655641,1.0,1.892791e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.000000e+00,1,False,False,1.0,7.900571,1.0,9.508861,1.0,1.000000e+00
60,False,0,0,1,0,1.0,1.000000e+00,0,False,True,1.0,7.353676,1.0,10.065557,1.0,1.000000e+00
61,False,0,0,1,1,1.0,1.000000e+00,0,False,True,1.0,8.019910,1.0,10.274669,1.0,1.000000e+00
62,False,0,0,1,2,1.0,1.000000e+00,0,False,True,1.0,7.814812,1.0,8.014084,1.0,1.000000e+00


In [257]:
(expectations_df['left_expectation'] - expectations_df['left_svd_exp']).describe()

count    64.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

In [258]:
(expectations_df['right_expectation'] - expectations_df['right_svd_exp']).describe()

count    64.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

In [259]:
expectations_df['left_jw_error'].describe()

count    64.000000
mean      8.691927
std       2.303631
min       2.816355
25%       8.338984
50%       9.496207
75%      10.127175
max      10.964077
Name: left_jw_error, dtype: float64

In [260]:
expectations_df['right_jw_error'].describe()

count    64.000000
mean      9.399459
std       1.474549
min       5.743955
25%       9.253907
50%       9.657611
75%      10.339312
max      11.273186
Name: right_jw_error, dtype: float64

In [261]:
expectations_df['left_jw_phase'] = expectations_df['left_jw_phase'].round()

In [262]:
expectations_df['left_jw_phase'].value_counts()

left_jw_phase
 1.0    40
-1.0    24
Name: count, dtype: int64

In [263]:
expectations_df['right_jw_phase'] = expectations_df['right_jw_phase'].round()

In [264]:
expectations_df['right_jw_phase'].round().value_counts()

right_jw_phase
 1.0    40
-1.0    24
Name: count, dtype: int64

In [265]:
(expectations_df['right_jw_phase']*expectations_df['left_jw_phase']).round().value_counts()

1.0    64
Name: count, dtype: int64

In [266]:
expectations_df[['predicted_jw_necessary', 'left_jw_phase', 'right_jw_phase']].value_counts()

predicted_jw_necessary  left_jw_phase  right_jw_phase
False                    1.0            1.0              40
True                    -1.0           -1.0              24
Name: count, dtype: int64

In [267]:
expectations_df['left_expectation'].describe()

count    64.000000
mean      0.999996
std       0.000017
min       0.999901
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: left_expectation, dtype: float64

In [268]:
(
    expectations_df[~expectations_df['jw_string_present_bool']]
)['left_expectation']

0     1.000000
1     1.000000
2     1.000000
3     1.000000
8     1.000000
9     1.000000
10    1.000000
11    1.000000
16    1.000000
17    0.999994
18    1.000000
19    1.000000
24    1.000000
25    1.000000
26    1.000000
27    1.000000
32    1.000000
33    1.000000
34    1.000000
35    1.000000
40    1.000000
41    0.999990
42    0.999999
43    1.000000
48    1.000000
49    0.999935
50    0.999998
51    0.999998
56    1.000000
57    1.000000
58    1.000000
59    1.000000
Name: left_expectation, dtype: float64

In [269]:
(
    expectations_df[expectations_df['jw_string_present_bool']]
)['left_expectation']

4     1.000000
5     1.000000
6     1.000000
7     1.000000
12    1.000000
13    1.000000
14    1.000000
15    1.000000
20    1.000000
21    0.999999
22    1.000000
23    1.000000
28    1.000000
29    1.000000
30    1.000000
31    1.000000
36    1.000000
37    1.000000
38    1.000000
39    1.000000
44    0.999989
45    1.000000
46    0.999901
47    1.000000
52    1.000000
53    0.999935
54    0.999998
55    0.999998
60    1.000000
61    1.000000
62    1.000000
63    1.000000
Name: left_expectation, dtype: float64

In [231]:
(
    expectations_df[~expectations_df['jw_string_present_bool']]
)['right_expectation']

0     1.000000e+00
1     8.350307e-14
2     8.339337e-14
3     1.000000e+00
8     1.000000e+00
9     1.828361e-14
10    1.484096e-14
11    3.263023e-14
16    1.000000e+00
17    2.250570e-14
18    1.173689e-14
19    1.687474e-14
24    1.000000e+00
25    1.000000e+00
26    1.269122e-14
27    1.269122e-14
32    1.000000e+00
33    3.946076e-14
34    1.000000e+00
35    3.948722e-14
40    1.000000e+00
41    9.912543e-16
42    1.941947e-14
43    7.802992e-15
48    1.000000e+00
49    3.211215e-15
50    1.485644e-14
51    6.391326e-15
56    1.000000e+00
57    1.000000e+00
58    1.000000e+00
59    1.000000e+00
Name: right_expectation, dtype: float64

In [92]:
expectations_df['right_expectation_rounded'] = expectations_df['right_expectation'].round()
expectations_df['left_expectation_rounded'] = expectations_df['left_expectation'].round()

In [93]:
expectations_df['correct'] = (
    (expectations_df['right_expectation_rounded'] == expectations_df['predicted_expectation']) &
    (expectations_df['left_expectation_rounded'] == expectations_df['predicted_expectation'])
)

In [94]:
expectations_df['correct'].value_counts()

correct
False    50
True     14
Name: count, dtype: int64

In [95]:
expectations_df[expectations_df['correct']]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,1.0,True
8,True,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
16,True,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
24,False,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
25,False,0,1,0,1,1.0,1.0,1,False,False,1.0,1.0,True
32,False,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
34,False,1,0,0,2,1.0,1.0,1,False,False,1.0,1.0,True
40,True,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
48,True,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True


In [96]:
expectations_df[
    (~expectations_df['correct'])
    & (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
60,False,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
61,False,0,0,1,1,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
62,False,0,0,1,2,1.000000,1.000000e+00,0,False,True,1.0,1.0,False


In [97]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
48,True,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
56,False,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
57,False,0,0,0,1,1.000000,1.000000e+00,1,False,False,1.0,1.0,True


In [98]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
    & (expectations_df['right_expectation_rounded'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False


# Sweep over on 4 boundary sites with gradients, iterate on both sides

## Sweep

In [330]:
block_width = 4
num_one_sided_iterations = 2
num_two_sided_iterations = 2
num_layers = 1
num_sites = 4

In [359]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()
left_phase_errors = dict()
right_phase_errors = dict()
right_grads = dict()
left_grads = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width

        for _ in range(num_two_sided_iterations):
            out = two_sided_optimise_layer_with_gradient(
                symmetry_transfer_matrix,
                top_left_mps_tensors[-1],
                top_right_mps_tensors[-1],
                right_side_left_schmidt_values_list[-1],
                left_side_right_schmidt_values_list[-1],
                block_width,
                block_offset,
                current_right_expectations,
                current_left_expectations,
                current_left_mps_tensors,
                current_right_mps_tensors,
                current_left_unitaries,
                current_right_unitaries,
                num_one_sided_iterations
            )
    
            left_triple, right_triple = out
        
            top_right_mps_tensors.append(right_triple[0])
            right_side_left_schmidt_values_list.append(right_triple[1])
            
            top_left_mps_tensors.append(left_triple[0])
            left_side_right_schmidt_values_list.append(left_triple[1])

        # Only works for one layer, one block unitaries...
        left_phase_error_pair = fermion_parity_operator_check(
            current_left_unitaries[-1][-1],
            left_boundary_jw
        )
        right_phase_error_pair = fermion_parity_operator_check(
            current_right_unitaries[-1][-1],
            right_boundary_jw
        )

    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries
    left_phase_errors[k] = left_phase_error_pair
    right_phase_errors[k] = right_phase_error_pair
    left_grads[k] = left_triple[2]
    right_grads[k] = right_triple[2]

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

In [360]:
right_expectations[(True, (0, 0), 0, 1)]

[[[3.211214886573137e-15], [3.211214886573137e-15]],
 [[0.9999999999999987], [0.9999999999999987]]]

## JW assumption check

In [361]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    right_grad = right_grads[k]
    _, rs, _ = npc.svd(right_grad)
    right_svd_exp = np.sum(rs)
    
    left_grad = left_grads[k]
    _, ls, _ = npc.svd(left_grad)
    left_svd_exp = np.sum(ls)

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[-1][-1][-1],
        vr[-1][-1][-1],
        exp,
        b1,
        b2,
        *left_phase_errors[k],
        *right_phase_errors[k],
        left_svd_exp,
        right_svd_exp
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool',
        'left_jw_phase',
        'left_jw_error',
        'right_jw_phase',
        'right_jw_error',
        'left_svd_exp',
        'right_svd_exp'
    ]
)

In [379]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,left_jw_phase,left_jw_error,right_jw_phase,right_jw_error,left_svd_exp,right_svd_exp
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,9.708713,1.0,9.966815,1.0,1.0
1,False,1,1,0,1,1.0,1.0,0,True,False,1.0,9.744570,1.0,9.978382,1.0,1.0
2,False,1,1,0,2,1.0,1.0,0,True,False,1.0,9.935628,1.0,10.405268,1.0,1.0
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,9.895252,1.0,9.747297,1.0,1.0
4,False,1,1,1,0,1.0,1.0,0,False,True,1.0,10.425921,1.0,10.094776,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.0,1,False,False,1.0,10.202371,1.0,10.101314,1.0,1.0
60,False,0,0,1,0,1.0,1.0,0,False,True,1.0,10.554164,1.0,10.702739,1.0,1.0
61,False,0,0,1,1,1.0,1.0,0,False,True,1.0,10.226371,1.0,10.222171,1.0,1.0
62,False,0,0,1,2,1.0,1.0,0,False,True,1.0,10.728897,1.0,11.076605,1.0,1.0


In [380]:
expectations_df['left_expectation'].describe()

count    6.400000e+01
mean     1.000000e+00
std      5.908173e-15
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: left_expectation, dtype: float64

In [381]:
expectations_df['right_expectation'].describe()

count    6.400000e+01
mean     1.000000e+00
std      5.892738e-15
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: right_expectation, dtype: float64

In [382]:
expectations_df['left_jw_phase'] = expectations_df['left_jw_phase'].round()

In [383]:
expectations_df['left_jw_phase'].value_counts()

left_jw_phase
1.0    64
Name: count, dtype: int64

In [384]:
expectations_df['right_jw_phase'] = expectations_df['right_jw_phase'].round()

In [385]:
expectations_df['right_jw_phase'].round().value_counts()

right_jw_phase
1.0    64
Name: count, dtype: int64

In [386]:
(expectations_df['right_jw_phase']*expectations_df['left_jw_phase']).round().value_counts()

1.0    64
Name: count, dtype: int64

Check unitaries

In [391]:
test_unitary = right_unitaries[(False, (0,0), 0, 0)][-1][-1]
test_unitary = test_unitary.split_legs()

In [392]:
test_unitary

<npc.Array shape=(4, 2, 4, 2, 4, 2, 4, 2) labels=['p0', 'p1', 'p2', 'p3', 'p0*', 'p1*', 'p2*', 'p3*']>

In [395]:
X = np.round(test_unitary.to_ndarray(), 2)

In [396]:
X

array([[[[[[[[ 0.98+0.j,  0.  +0.j],
             [-0.06+0.j,  0.  +0.j],
             [ 0.05+0.j,  0.  +0.j],
             [ 0.02+0.j,  0.  +0.j]],

            [[ 0.  +0.j, -0.  +0.j],
             [ 0.  +0.j, -0.  +0.j],
             [-0.  +0.j, -0.  +0.j],
             [-0.  +0.j,  0.  +0.j]]],


           [[[-0.03+0.j,  0.  +0.j],
             [-0.  +0.j, -0.  +0.j],
             [-0.05+0.j,  0.  +0.j],
             [-0.03+0.j, -0.  +0.j]],

            [[-0.  +0.j, -0.  +0.j],
             [ 0.  +0.j,  0.  +0.j],
             [-0.  +0.j,  0.  +0.j],
             [ 0.  +0.j,  0.  +0.j]]],


           [[[-0.09+0.j,  0.  +0.j],
             [ 0.05+0.j, -0.  +0.j],
             [ 0.01+0.j, -0.  +0.j],
             [ 0.02+0.j,  0.  +0.j]],

            [[-0.  +0.j,  0.  +0.j],
             [ 0.  +0.j,  0.  +0.j],
             [ 0.  +0.j, -0.  +0.j],
             [ 0.  +0.j,  0.  +0.j]]],


           [[[ 0.11+0.j,  0.  +0.j],
             [ 0.02+0.j,  0.  +0.j],
             [-0.02+

# Sweep over on 3 boundary sites with gradients, iterate on both sides

In [405]:
symmetric_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                [s[1],] + s*49,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I, np_00],
                right_boundary_unitaries=[np_00, np_I, np_00]
            )

            symmetric_cases[(*k, i, j)] = case

In [406]:
for c in symmetric_cases.values():
    c.compute_svd_approximate_expectation()

In [410]:
leg_labels = ['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']

In [414]:
X = np_00[(...,)  + (np.newaxis,)*2]*np_JW[(np.newaxis,)*2 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_00[(np.newaxis,)*4 + (...,)]

In [417]:
right_boundary_jw_3 = npc.Array.from_ndarray(
    X,
    legs_charges[:-2],
    dtype=np.complex128,
    labels=leg_labels
)

right_boundary_jw_3 = right_boundary_jw_3.combine_legs(
    [['p0', 'p1', 'p2'], ['p0*', 'p1*', 'p2*']]
)

In [420]:
def fermion_parity_operator_check_3(operator):
    op = (
        operator
        .transpose(['(p0.p1.p2)', '(p0*.p1*.p2*)'])
        .to_ndarray()
    )
    fp = right_boundary_jw_3.to_ndarray()

    transformed_op = fp @ op @ fp

    x = np.trace(transformed_op @ (op.T.conj()))
    phase = x/np.abs(x)

    assert np.isreal(phase)
    phase = np.real(phase)

    error = np.linalg.norm(
        phase*transformed_op - op,
        ord='fro'
    )

    return (phase, error)

## Sweep

In [421]:
block_width = 3
num_one_sided_iterations = 2
num_two_sided_iterations = 2
num_layers = 1
num_sites = 3

In [422]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()
left_phase_errors = dict()
right_phase_errors = dict()
right_grads = dict()
left_grads = dict()

for k, c in symmetric_cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width

        for _ in range(num_two_sided_iterations):
            out = two_sided_optimise_layer_with_gradient(
                symmetry_transfer_matrix,
                top_left_mps_tensors[-1],
                top_right_mps_tensors[-1],
                right_side_left_schmidt_values_list[-1],
                left_side_right_schmidt_values_list[-1],
                block_width,
                block_offset,
                current_right_expectations,
                current_left_expectations,
                current_left_mps_tensors,
                current_right_mps_tensors,
                current_left_unitaries,
                current_right_unitaries,
                num_one_sided_iterations
            )
    
            left_triple, right_triple = out
        
            top_right_mps_tensors.append(right_triple[0])
            right_side_left_schmidt_values_list.append(right_triple[1])
            
            top_left_mps_tensors.append(left_triple[0])
            left_side_right_schmidt_values_list.append(left_triple[1])

        # Only works for one layer, one block unitaries...
        left_phase_error_pair = fermion_parity_operator_check_3(
            current_left_unitaries[-1][-1],
        )
        right_phase_error_pair = fermion_parity_operator_check_3(
            current_right_unitaries[-1][-1]
        )

    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries
    left_phase_errors[k] = left_phase_error_pair
    right_phase_errors[k] = right_phase_error_pair
    left_grads[k] = left_triple[2]
    right_grads[k] = right_triple[2]

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

## JW assumption check

In [423]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    right_grad = right_grads[k]
    _, rs, _ = npc.svd(right_grad)
    right_svd_exp = np.sum(rs)
    
    left_grad = left_grads[k]
    _, ls, _ = npc.svd(left_grad)
    left_svd_exp = np.sum(ls)

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[-1][-1][-1],
        vr[-1][-1][-1],
        exp,
        b1,
        b2,
        *left_phase_errors[k],
        *right_phase_errors[k],
        left_svd_exp,
        right_svd_exp
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool',
        'left_jw_phase',
        'left_jw_error',
        'right_jw_phase',
        'right_jw_error',
        'left_svd_exp',
        'right_svd_exp'
    ]
)

In [424]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,left_jw_phase,left_jw_error,right_jw_phase,right_jw_error,left_svd_exp,right_svd_exp
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,7.306835,1.0,7.508454,1.0,1.0
1,False,1,1,0,1,1.0,1.0,0,True,False,1.0,7.546617,1.0,7.177885,1.0,1.0
2,False,1,1,0,2,1.0,1.0,0,True,False,1.0,7.201555,1.0,7.093977,1.0,1.0
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,6.899593,1.0,7.427641,1.0,1.0
4,False,1,1,1,0,1.0,1.0,0,False,True,1.0,7.259854,1.0,6.822480,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.0,1,False,False,1.0,7.618837,1.0,7.364202,1.0,1.0
60,False,0,0,1,0,1.0,1.0,0,False,True,1.0,7.433756,1.0,7.100573,1.0,1.0
61,False,0,0,1,1,1.0,1.0,0,False,True,1.0,7.451012,1.0,6.896124,1.0,1.0
62,False,0,0,1,2,1.0,1.0,0,False,True,1.0,7.272620,1.0,6.643666,1.0,1.0


In [425]:
expectations_df['left_expectation'].describe()

count    6.400000e+01
mean     1.000000e+00
std      5.936781e-15
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: left_expectation, dtype: float64

In [426]:
expectations_df['right_expectation'].describe()

count    6.400000e+01
mean     1.000000e+00
std      6.045803e-15
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: right_expectation, dtype: float64

In [427]:
expectations_df['left_jw_phase'] = expectations_df['left_jw_phase'].round()

In [428]:
expectations_df['left_jw_phase'].value_counts()

left_jw_phase
1.0    64
Name: count, dtype: int64

In [429]:
expectations_df['right_jw_phase'] = expectations_df['right_jw_phase'].round()

In [430]:
expectations_df['right_jw_phase'].round().value_counts()

right_jw_phase
1.0    64
Name: count, dtype: int64

In [431]:
(expectations_df['right_jw_phase']*expectations_df['left_jw_phase']).round().value_counts()

1.0    64
Name: count, dtype: int64

Check unitaries

In [432]:
test_unitary = right_unitaries[(False, (0,0), 0, 0)][-1][-1]
test_unitary = test_unitary.split_legs()

In [433]:
test_unitary

<npc.Array shape=(4, 2, 4, 4, 2, 4) labels=['p0', 'p1', 'p2', 'p0*', 'p1*', 'p2*']>

In [434]:
X = np.round(test_unitary.to_ndarray(), 2)

In [435]:
X

array([[[[[[ 0.98+0.j,  0.02+0.j,  0.05+0.j,  0.09+0.j],
           [-0.  +0.j,  0.  +0.j,  0.  +0.j, -0.  +0.j]],

          [[-0.04+0.j,  0.01+0.j, -0.05+0.j,  0.02+0.j],
           [ 0.  +0.j,  0.  +0.j, -0.  +0.j, -0.  +0.j]],

          [[-0.02+0.j,  0.03+0.j, -0.03+0.j,  0.05+0.j],
           [-0.  +0.j, -0.  +0.j,  0.  +0.j,  0.  +0.j]],

          [[-0.06+0.j, -0.09+0.j,  0.01+0.j,  0.  +0.j],
           [ 0.  +0.j,  0.  +0.j,  0.  +0.j, -0.  +0.j]]],


         [[[ 0.09+0.j,  0.3 +0.j, -0.19+0.j, -0.1 +0.j],
           [-0.01+0.j, -0.01+0.j, -0.01+0.j, -0.01+0.j]],

          [[-0.05+0.j, -0.07+0.j,  0.56+0.j,  0.14+0.j],
           [-0.01+0.j, -0.01+0.j, -0.01+0.j, -0.01+0.j]],

          [[ 0.12+0.j, -0.49+0.j,  0.22+0.j, -0.09+0.j],
           [-0.01+0.j, -0.01+0.j, -0.01+0.j, -0.01+0.j]],

          [[-0.  +0.j,  0.3 +0.j, -0.07+0.j,  0.33+0.j],
           [-0.01+0.j, -0.01+0.j, -0.01+0.j, -0.01+0.j]]],


         [[[-0.01+0.j, -0.15+0.j,  0.13+0.j,  0.36+0.j],
           

# Conclusion
Something wrong. Rather than relying on previous functions, will step through counter examples explicitly in a future notebook.

# Old code

## Sweep over on 6 boundary sites

In [98]:
block_width = 6
num_iterations = 2
num_layers = 1
num_sites = 6

In [99]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])


    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

### JW assumption check

In [82]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [83]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((2.1610706909084384e-17, 1, True, True),
 (8.350306722905389e-14, 0, True, False))

In [84]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [85]:
list(left_expectations.keys())[0]

(False, (1, 1), 0, 0)

In [86]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[0][0][0],
        vr[0][0][0],
        exp,
        b1,
        b2
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool'
    ]
)

In [87]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool
0,False,1,1,0,0,1.0,1.000000e+00,1,False,False
1,False,1,1,0,1,1.0,8.350307e-14,0,True,False
2,False,1,1,0,2,1.0,8.339337e-14,0,True,False
3,False,1,1,0,3,1.0,1.000000e+00,1,False,False
4,False,1,1,1,0,1.0,1.892791e-14,0,False,True
...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.000000e+00,1,False,False
60,False,0,0,1,0,1.0,1.000000e+00,0,False,True
61,False,0,0,1,1,1.0,1.000000e+00,0,False,True
62,False,0,0,1,2,1.0,1.000000e+00,0,False,True


In [88]:
expectations_df['left_expectation']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
     ... 
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
Name: left_expectation, Length: 64, dtype: float64

In [89]:
expectations_df['left_expectation'].describe()

count    64.000000
mean      0.999996
std       0.000017
min       0.999901
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: left_expectation, dtype: float64

In [90]:
expectations_df['right_expectation_rounded'] = expectations_df['right_expectation'].round()
expectations_df['left_expectation_rounded'] = expectations_df['left_expectation'].round()

In [91]:
expectations_df['correct'] = (
    (expectations_df['right_expectation_rounded'] == expectations_df['predicted_expectation']) &
    (expectations_df['left_expectation_rounded'] == expectations_df['predicted_expectation'])
)

In [92]:
expectations_df['correct'].value_counts()

correct
False    50
True     14
Name: count, dtype: int64

In [93]:
expectations_df[expectations_df['correct']]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,1.0,True
8,True,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
16,True,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
24,False,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
25,False,0,1,0,1,1.0,1.0,1,False,False,1.0,1.0,True
32,False,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
34,False,1,0,0,2,1.0,1.0,1,False,False,1.0,1.0,True
40,True,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
48,True,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True


In [94]:
expectations_df[
    (~expectations_df['correct'])
    & (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
60,False,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
61,False,0,0,1,1,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
62,False,0,0,1,2,1.000000,1.000000e+00,0,False,True,1.0,1.0,False


In [96]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
48,True,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False
56,False,0,0,0,0,1.000000,1.000000e+00,1,False,False,1.0,1.0,True
57,False,0,0,0,1,1.000000,1.000000e+00,1,False,False,1.0,1.0,True


In [97]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
    & (expectations_df['right_expectation_rounded'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,0.999935,3.211215e-15,1,False,False,0.0,1.0,False
50,True,0,0,0,2,0.999998,1.485644e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,0.999998,6.391326e-15,1,False,False,0.0,1.0,False
53,True,0,0,1,1,0.999935,3.211215e-15,0,False,True,0.0,1.0,False
54,True,0,0,1,2,0.999998,1.485644e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,0.999998,6.391326e-15,0,False,True,0.0,1.0,False


Still a problem at 6 sites, so sign of a more general issue.

## Sweep over on 4 boundary sites, shift sites by 1.

In [111]:
block_width = 4
num_iterations = 1
num_layers = 1
num_sites = 4

In [112]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()

for k, c in shifted_cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])


    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (True, (1, 1), 0, 0)
Case (True, (1, 1), 0, 1)
Case (True, (1, 1), 0, 2)
Case (True, (1, 1), 0, 3)
Case (True, (1, 1), 1, 0)
Case (True, (1, 1), 1, 1)
Case (True, (1, 1), 1, 2)
Case (True, (1, 1), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (False, (0, 1), 0, 0)
Case (False, (0, 1), 0, 1)
Case (False, (0, 1), 0, 2)
Case (False, (0, 1), 0, 3)
Case (False, (0, 1), 1, 0)
Case (False, (0, 1), 1, 1)
Case (False, (0, 1), 1, 2)
Case (False, (0, 1), 1, 3)
Case (False, (1, 0), 0, 0)
Case (False, (1, 0), 0, 1)
Case (False, (1, 0), 0, 2)
Case (False, (1, 0), 0, 3)
Case (False, (1, 0), 1, 0)
Case (False, (1, 

### JW assumption check

In [113]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [114]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((1.6501719539551946e-17, 1, True, True),
 (8.359186027048973e-14, 0, True, False))

In [115]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [116]:
list(left_expectations.keys())[0]

(False, (1, 1), 0, 0)

In [117]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[0][0][0],
        vr[0][0][0],
        exp,
        b1,
        b2
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool'
    ]
)

In [118]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool
0,False,1,1,0,0,1.0,1.000000e+00,1,False,False
1,False,1,1,0,1,1.0,8.359186e-14,0,True,False
2,False,1,1,0,2,1.0,8.321829e-14,0,True,False
3,False,1,1,0,3,1.0,1.000000e+00,1,False,False
4,False,1,1,1,0,1.0,1.891021e-14,0,False,True
...,...,...,...,...,...,...,...,...,...,...
59,False,0,0,0,3,1.0,1.000000e+00,1,False,False
60,False,0,0,1,0,1.0,1.000000e+00,0,False,True
61,False,0,0,1,1,1.0,1.000000e+00,0,False,True
62,False,0,0,1,2,1.0,1.000000e+00,0,False,True


In [119]:
expectations_df['left_expectation']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
     ... 
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
Name: left_expectation, Length: 64, dtype: float64

In [120]:
expectations_df['left_expectation'].describe()

count    64.000000
mean      0.999999
std       0.000006
min       0.999952
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: left_expectation, dtype: float64

In [121]:
expectations_df['right_expectation_rounded'] = expectations_df['right_expectation'].round()
expectations_df['left_expectation_rounded'] = expectations_df['left_expectation'].round()

In [122]:
expectations_df['correct'] = (
    (expectations_df['right_expectation_rounded'] == expectations_df['predicted_expectation']) &
    (expectations_df['left_expectation_rounded'] == expectations_df['predicted_expectation'])
)

In [123]:
expectations_df['correct'].value_counts()

correct
False    50
True     14
Name: count, dtype: int64

In [124]:
expectations_df[expectations_df['correct']]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,1.0,True
8,True,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
16,True,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
24,False,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
25,False,0,1,0,1,1.0,1.0,1,False,False,1.0,1.0,True
32,False,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
34,False,1,0,0,2,1.0,1.0,1,False,False,1.0,1.0,True
40,True,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
48,True,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True


In [125]:
expectations_df[
    (~expectations_df['correct'])
    & (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,1.0,2.694831e-14,1,False,False,0.0,1.0,False
50,True,0,0,0,2,1.0,2.163729e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,1.0,4.375476e-14,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.0,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,1.0,2.694831e-14,0,False,True,0.0,1.0,False
54,True,0,0,1,2,1.0,2.163729e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,1.0,4.375476e-14,0,False,True,0.0,1.0,False
60,False,0,0,1,0,1.0,1.000000e+00,0,False,True,1.0,1.0,False
61,False,0,0,1,1,1.0,1.000000e+00,0,False,True,1.0,1.0,False
62,False,0,0,1,2,1.0,1.000000e+00,0,False,True,1.0,1.0,False


In [126]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
48,True,0,0,0,0,1.0,1.000000e+00,1,False,False,1.0,1.0,True
49,True,0,0,0,1,1.0,2.694831e-14,1,False,False,0.0,1.0,False
50,True,0,0,0,2,1.0,2.163729e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,1.0,4.375476e-14,1,False,False,0.0,1.0,False
52,True,0,0,1,0,1.0,1.000000e+00,0,False,True,1.0,1.0,False
53,True,0,0,1,1,1.0,2.694831e-14,0,False,True,0.0,1.0,False
54,True,0,0,1,2,1.0,2.163729e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,1.0,4.375476e-14,0,False,True,0.0,1.0,False
56,False,0,0,0,0,1.0,1.000000e+00,1,False,False,1.0,1.0,True
57,False,0,0,0,1,1.0,1.000000e+00,1,False,False,1.0,1.0,True


In [127]:
expectations_df[
    (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
    & (expectations_df['right_expectation_rounded'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
49,True,0,0,0,1,1.0,2.694831e-14,1,False,False,0.0,1.0,False
50,True,0,0,0,2,1.0,2.163729e-14,1,False,False,0.0,1.0,False
51,True,0,0,0,3,1.0,4.375476e-14,1,False,False,0.0,1.0,False
53,True,0,0,1,1,1.0,2.694831e-14,0,False,True,0.0,1.0,False
54,True,0,0,1,2,1.0,2.163729e-14,0,False,True,0.0,1.0,False
55,True,0,0,1,3,1.0,4.375476e-14,0,False,True,0.0,1.0,False


In [88]:
(
    expected_left_expecations[(False, (1, 1), 1, 1)],
    expected_left_expecations[(False, (1, 1), 0, 1)]
)

((1.0000000000000038, 1, True, True), (0.9999999282052796, 0, True, False))

In [89]:
expected_left_expecations

{(False, (1, 1), 0, 0): (1.000000000000004, 1, False, False),
 (False, (1, 1), 0, 1): (0.9999999282052796, 0, True, False),
 (False, (1, 1), 0, 2): (0.9999999995360392, 0, True, False),
 (False, (1, 1), 0, 3): (1.0000000000000049, 1, False, False),
 (False, (1, 1), 1, 0): (0.9999999935589872, 0, False, True),
 (False, (1, 1), 1, 1): (1.0000000000000038, 1, True, True),
 (False, (1, 1), 1, 2): (1.0000000000000047, 1, True, True),
 (False, (1, 1), 1, 3): (0.9999999651742371, 0, False, True),
 (False, (0, 0), 0, 0): (1.000000000000009, 1, False, False),
 (False, (0, 0), 0, 1): (1.0000000000000087, 1, False, False),
 (False, (0, 0), 0, 2): (1.0000000000000056, 1, False, False),
 (False, (0, 0), 0, 3): (1.0000000000000053, 1, False, False),
 (False, (0, 0), 1, 0): (1.000000000000009, 0, False, True),
 (False, (0, 0), 1, 1): (1.0000000000000087, 0, False, True),
 (False, (0, 0), 1, 2): (1.0000000000000056, 0, False, True),
 (False, (0, 0), 1, 3): (1.0000000000000053, 0, False, True),
 (True,